# Finite-sample simulation (Python)

This notebook runs a minimal Monte Carlo simulation to illustrate finite-sample behavior of spatial (Conley/HAC) standard errors for non-linear models:

- logit
- probit
- poisson
- negative binomial (NB2)

Data-generating process (DGP): we use a **Gaussian copula** to introduce spatial dependence *while keeping the correct marginal model for each observation*. That way `beta_true` is the right target parameter, and any inference issues come from spatial dependence (not misspecification).

For each model we compare:

- non-spatial robust SE (HC1)
- Conley SE (Bartlett kernel)
- Conley SE (Uniform/rectangle kernel)

Output: a compact summary table and `simulation_summary.csv`.

In [18]:
import math
import warnings
from dataclasses import dataclass

import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm

import importlib
import spacereg
importlib.reload(spacereg)
SpatialStandardErrorsComputer = spacereg.SpatialStandardErrorsComputer

warnings.filterwarnings('ignore')
np.set_printoptions(precision=4, suppress=True)
pd.set_option('display.width', 140)
pd.set_option('display.max_columns', 60)


In [13]:
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1.0 / (1.0 + np.exp(-x))


def make_grid(side: int) -> tuple[np.ndarray, np.ndarray]:
    # Integer grid coordinates 1..side.
    c1 = np.repeat(np.arange(1, side + 1), side).astype(float)
    c2 = np.tile(np.arange(1, side + 1), side).astype(float)
    return c1, c2


def pairwise_dist(c1: np.ndarray, c2: np.ndarray) -> np.ndarray:
    dx = c1[:, None] - c1[None, :]
    dy = c2[:, None] - c2[None, :]
    return np.sqrt(dx * dx + dy * dy)


def gaussian_copula_uniform(c1: np.ndarray, c2: np.ndarray, phi: float, rng: np.random.Generator) -> np.ndarray:
    """Generate spatially dependent U(0,1) draws via a Gaussian copula.

    Correlation is exp(-d/phi). Each margin is uniform, so the fitted model's
    marginal likelihood is correctly specified; dependence is only across i.
    """
    d = pairwise_dist(c1, c2)
    R = np.exp(-d / phi)
    R = R + np.eye(R.shape[0]) * 1e-10  # jitter
    L = np.linalg.cholesky(R)
    z = L @ rng.standard_normal(R.shape[0])
    u = st.norm.cdf(z)
    u = np.clip(u, 1e-12, 1.0 - 1e-12)
    return u


def build_dataframe(y: np.ndarray, x: np.ndarray, c1: np.ndarray, c2: np.ndarray, cutoff: float) -> pd.DataFrame:
    # spacereg expects: y column, x columns, coordinate columns, and cutoff columns.
    return pd.DataFrame({
        'y': y.astype(float),
        'indep1': x.astype(float),
        'const': 1.0,
        'C1': c1.astype(float),
        'C2': c2.astype(float),
        'cutoff1': float(cutoff),
        'cutoff2': float(cutoff),
    })


In [25]:
@dataclass(frozen=True)
class SimConfig:
    # grid size: N = side^2
    side: int = 10
    # Monte Carlo repetitions
    reps: int = 200
    seed: int = 12345
    # true coefficient on indep1
    beta_true: float = 0.5
    # spatial dependence length (Gaussian copula correlation exp(-d/phi))
    phi: float = 1.0
    # Conley cutoff (grid distance)
    cutoff: float = 3.0
    # NB2 overdispersion Var = mu + alpha*mu^2
    nb_alpha: float = 0.8

cfg = SimConfig()
cfg


SimConfig(side=10, reps=200, seed=12345, beta_true=0.5, phi=1.0, cutoff=3.0, nb_alpha=0.8)

In [26]:
def fit_beta_and_hc1_se(model_name: str, df: pd.DataFrame) -> tuple[float, float]:
    y = df['y'].to_numpy()
    X = df[['indep1', 'const']].to_numpy()

    if model_name == 'logit':
        res = sm.Logit(y, X).fit(disp=0, cov_type='HC1')
    elif model_name == 'probit':
        res = sm.Probit(y, X).fit(disp=0, cov_type='HC1')
    elif model_name == 'poisson':
        res = sm.GLM(y, X, family=sm.families.Poisson()).fit(disp=0, cov_type='HC1')
    elif model_name == 'nb':
        # Keep alpha fixed for stability.
        res = sm.GLM(y, X, family=sm.families.NegativeBinomial(alpha=cfg.nb_alpha)).fit(disp=0, cov_type='HC1')
    else:
        raise ValueError(f'Unknown model: {model_name}')

    beta_hat = float(res.params[0])
    se_hc1 = float(res.bse[0])
    return beta_hat, se_hc1


def conley_se(model_name: str, df: pd.DataFrame, kernel: str) -> float:
    base = SpatialStandardErrorsComputer(df, coordinates=['C1', 'C2'], cutoffs=['cutoff1', 'cutoff2'])
    se_series = base.compute_conley_standard_errors_all_models(model_name, 'y', ['indep1', 'const'], kernel=kernel)
    return float(se_series.loc['indep1'])


def simulate_one(model_name: str, rng: np.random.Generator) -> dict:
    c1, c2 = make_grid(cfg.side)
    n = c1.shape[0]
    x = rng.standard_normal(n)

    # Spatial dependence via Gaussian copula: correlated uniforms.
    u = gaussian_copula_uniform(c1, c2, phi=cfg.phi, rng=rng)

    eta = cfg.beta_true * x

    if model_name == 'logit':
        p = sigmoid(eta)
        p = np.clip(p, 1e-12, 1.0 - 1e-12)
        y = (u < p).astype(float)
    elif model_name == 'probit':
        p = st.norm.cdf(eta)
        p = np.clip(p, 1e-12, 1.0 - 1e-12)
        y = (u < p).astype(float)
    elif model_name == 'poisson':
        mu = np.exp(eta)
        mu = np.clip(mu, 1e-12, 1e6)
        y = st.poisson(mu).ppf(u).astype(float)
    elif model_name == 'nb':
        mu = np.exp(eta)
        mu = np.clip(mu, 1e-12, 1e6)
        alpha = cfg.nb_alpha
        n_param = 1.0 / alpha
        p_param = n_param / (n_param + mu)
        y = st.nbinom(n_param, p_param).ppf(u).astype(float)
    else:
        raise ValueError(f'Unknown model: {model_name}')

    df = build_dataframe(y=y, x=x, c1=c1, c2=c2, cutoff=cfg.cutoff)

    beta_hat, se_hc1 = fit_beta_and_hc1_se(model_name, df)
    se_cb = conley_se(model_name, df, kernel='bartlett')
    se_cu = conley_se(model_name, df, kernel='uniform')

    z = 1.959963984540054  # ~N(0,1) 97.5% quantile
    cover_hc1 = float((beta_hat - z * se_hc1 <= cfg.beta_true) and (cfg.beta_true <= beta_hat + z * se_hc1))
    cover_cb = float((beta_hat - z * se_cb <= cfg.beta_true) and (cfg.beta_true <= beta_hat + z * se_cb))
    cover_cu = float((beta_hat - z * se_cu <= cfg.beta_true) and (cfg.beta_true <= beta_hat + z * se_cu))

    return {
        'model': model_name,
        'beta_hat': beta_hat,
        'se_hc1': se_hc1,
        'se_conley_bartlett': se_cb,
        'se_conley_uniform': se_cu,
        'cover_hc1': cover_hc1,
        'cover_conley_bartlett': cover_cb,
        'cover_conley_uniform': cover_cu,
    }


# quick smoke test
rng = np.random.default_rng(cfg.seed)
simulate_one('logit', rng)


{'model': 'logit',
 'beta_hat': 0.6693269180751575,
 'se_hc1': 0.2520063384084991,
 'se_conley_bartlett': 0.2518755512110301,
 'se_conley_uniform': 0.24384101912848608,
 'cover_hc1': 1.0,
 'cover_conley_bartlett': 1.0,
 'cover_conley_uniform': 1.0}

In [27]:
def run_mc(model_name: str) -> pd.DataFrame:
    rng = np.random.default_rng(cfg.seed)
    rows = []
    failures = 0
    for _ in range(cfg.reps):
        try:
            rows.append(simulate_one(model_name, rng))
        except Exception:
            failures += 1
            continue
    out = pd.DataFrame(rows)
    print(f'{model_name}: kept {len(out)}/{cfg.reps} reps (failures={failures})')
    return out

def summarize(df: pd.DataFrame) -> pd.Series:
    return pd.Series({
        'reps_used': len(df),
        'mean_beta_hat': df['beta_hat'].mean(),
        'emp_sd_beta_hat': df['beta_hat'].std(ddof=1),
        'mean_se_hc1': df['se_hc1'].mean(),
        'mean_se_conley_bartlett': df['se_conley_bartlett'].mean(),
        'mean_se_conley_uniform': df['se_conley_uniform'].mean(),
        'cover95_hc1': df['cover_hc1'].mean(),
        'cover95_conley_bartlett': df['cover_conley_bartlett'].mean(),
        'cover95_conley_uniform': df['cover_conley_uniform'].mean(),
    })

models = ['logit', 'probit', 'poisson', 'nb']
mc = {m: run_mc(m) for m in models}
summary = pd.DataFrame({m: summarize(mc[m]) for m in models}).T
summary

logit: kept 200/200 reps (failures=0)
probit: kept 200/200 reps (failures=0)
poisson: kept 200/200 reps (failures=0)
nb: kept 200/200 reps (failures=0)


,reps_used,mean_beta_hat,emp_sd_beta_hat,mean_se_hc1,mean_se_conley_bartlett,mean_se_conley_uniform,cover95_hc1,cover95_conley_bartlett,cover95_conley_uniform
logit,200.0,0.552445,0.227107,0.228387,0.219584,0.203045,0.950,0.94,0.865
probit,200.0,0.521422,0.144737,0.146703,0.140484,0.130206,0.970,0.96,0.875
poisson,200.0,0.500141,0.105405,0.094067,0.092472,0.087806,0.900,0.90,0.830
nb,200.0,0.494249,0.147888,0.134677,0.127713,0.114395,0.935,0.88,0.815


In [28]:
out_path = 'simulation_summary.csv'
summary.to_csv(out_path, index=True)
out_path

'simulation_summary.csv'